In [86]:
import csv
import codecs
import subprocess
import os
import re
import glob

species_names = []
species_dict_by_name = dict()
with codecs.open("DiwanSupplementOriginal.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        diwan_anti_sd = data[8][1:-1].upper()
        diwan_anti_sd = diwan_anti_sd.replace('U', 'T')
        revcompl = lambda x: ''.join([{'A':'T','C':'G','G':'C','T':'A'}[B] for B in x][::-1])
        diwan_sd = revcompl(diwan_anti_sd)
        #if (diwan_sd != "AGGAGG"):
         #   print(name, " ", diwan_sd)
        species_names.append(name)
        species_dict_by_name[name] = dict()
        species_dict_by_name[name]["diwan_sd"] = diwan_sd
        species_dict_by_name[name]["found_data_files"] = False
        species_dict_by_name[name]["assebly_file_prefix"] = ""
        species_dict_by_name[name]["16S_end"] = ""
        
        
# read the large excel file, and find the correct genome
with codecs.open("/scratch4/moamin/ncbi_bacteria_db/general_helix45_16s18_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        if (name in species_dict_by_name):
            species_dict_by_name[name]["16S_end"] = data[23]
            prefix = data[0]
            species_dict_by_name[name]["assebly_file_prefix"] = prefix
            species_dict_by_name[name]["found_data_files"] = True

count = 0
with open('DiwanDataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # count missing names, and get the full file path, output everything to a csv file
    for name in species_names:
        if (species_dict_by_name[name]["found_data_files"] == False):
            #print("Not Found for " + name)
            count += 1
        else:
            prefix = species_dict_by_name[name]["assebly_file_prefix"]
            #files = [f for f in os.listdir('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/') if re.match(r'*.gz', f)]
            names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, species_dict_by_name[name]["16S_end"], species_dict_by_name[name]["diwan_sd"]])
        
# getting 236 out of 285 species (found no matches for 48 species)
print("Not found matches for ", count, " species")

Not found matches for  48  species
